In [1]:
import random  # for reproducibility in sampling and shuffling
from pathlib import Path  # filesystem path manipulation

# 2. Data handling and preprocessing
import numpy as np  # numerical operations
import pandas as pd  # dataframes and tabular data

# 3. Visualization
import matplotlib.pyplot as plt  # plotting
import seaborn as sns  # statistical data visualization

# 4. Deep learning (PyTorch)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# 5. Machine learning utilities (scikit-learn)
from sklearn.preprocessing import LabelEncoder, StandardScaler  # encoding labels and feature scaling
from sklearn.model_selection import train_test_split  # data splitting
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score  # evaluation metrics

# 6. Learning rate schedulers
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR, OneCycleLR  # dynamic LR scheduling

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Configure seaborn aesthetics
sns.set(style='whitegrid', context='notebook')

In [2]:
# Dataset file paths
TRAIN_CSV = "/kaggle/input/har-dataset-2/train_data_2.csv"  # Path to training data CSV
TEST_CSV  = "/kaggle/input/har-dataset-2/test_data_2.csv"   # Path to testing data CSV

# Windowing and batching parameters
WINDOW_SIZE = 600           # Number of timesteps per sliding window
BATCH_SIZE = 32             # Number of samples per training batch
SAMPLING_FREQ = 200         # Data sampling frequency in Hz
ADD_FEATS = False           # Whether to add derived features beyond raw sensor data

# Detect compute device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Processing Utilities and Dataset Preparation
This cell defines utility functions to select sensor channels, split data into training and validation sets, and loads, preprocesses, encodes, and wraps data in PyTorch datasets and dataloaders.

In [3]:
class DataPreprocessor:
    def __init__(self, window_size=600, sampling_freq=200, target_samples=32):
        self.window_size = window_size
        self.sampling_freq = sampling_freq
        self.target_samples = target_samples

    def moving_average_filter(self, data, window=8):
        return data.rolling(window=window, center=True).mean().fillna(data)

    def segment_data(self, data, overlap=0.7):
        segments = []
        labels_io = []
        labels_scene = []
        step_size = int(self.window_size * (1 - overlap))

        for i in range(0, len(data) - self.window_size, step_size):
            window = data.iloc[i:i+self.window_size]

            if len(window['I/O Detection'].unique()) == 1 and \
               len(window['Scene Identification'].unique()) == 1:
                segments.append(window)
                labels_io.append(window['I/O Detection'].iloc[0])
                labels_scene.append(window['Scene Identification'].iloc[0])

        return segments, labels_io, labels_scene

    def downsample_segment(self, segment, sensor_cols):
        downsampled = {}
        for col in sensor_cols:
            if col in segment.columns:
                original_data = segment[col].values
                indices = np.linspace(0, len(original_data) - 1, self.target_samples, dtype=int)
                downsampled[col] = original_data[indices]
            else:
                downsampled[col] = np.zeros(self.target_samples)
        return downsampled

    def z_score_normalize(self, segment_data):
        segment_array = np.array(list(segment_data.values())).T
        mean = np.mean(segment_array, axis=0)
        std = np.std(segment_array, axis=0)
        std = np.where(std == 0, 1, std)
        return (segment_array - mean) / std

    def extract_statistical_features(self, segment, sensor_cols):
        features = []
        for col in sensor_cols:
            if col in segment.columns:
                values = segment[col].values
                features.extend([
                    np.mean(values),
                    np.std(values),
                    np.min(values),
                    np.max(values)
                ])
        return features

    def preprocess_data(self, data, sensor_cols):
        filtered_data = data.copy()
        for col in sensor_cols:
            if col in data.columns:
                filtered_data[col] = self.moving_average_filter(data[col])

        segments, labels_io, labels_scene = self.segment_data(filtered_data)

        processed_segments = []
        statistical_features = []

        for segment in segments:
            if self.target_samples is not None:
                downsampled = self.downsample_segment(segment, sensor_cols)
                normalized = self.z_score_normalize(downsampled)
                processed_segments.append(normalized)
            else:
                processed_segments.append(segment[sensor_cols].values)

            stats = self.extract_statistical_features(segment, sensor_cols)
            statistical_features.append(stats)

        return np.array(processed_segments), np.array(statistical_features), labels_io, labels_scene

class HARDSdataset(Dataset):
    def __init__(self, X, y_io, y_scene, io_label_encoder, scene_label_encoder, statistical_features=None):
        self.X = torch.FloatTensor(X)
        self.y_io = torch.LongTensor(y_io)
        self.y_scene = torch.LongTensor(y_scene)
        self.statistical_features = statistical_features
        if statistical_features is not None:
            self.statistical_features = torch.FloatTensor(statistical_features)
        self.io_label_encoder = io_label_encoder
        self.scene_label_encoder = scene_label_encoder
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.statistical_features is not None:
            return self.X[idx], self.y_io[idx], self.y_scene[idx], self.statistical_features[idx]
        return self.X[idx], self.y_io[idx], self.y_scene[idx]

In [4]:
# Function to map high-level sensor names to their corresponding DataFrame columns
def get_sensors(sensor_list):
    # Dictionary of available sensor groups and their DataFrame columns
    available_sensors = {
        "Accelerometer": ["A_x [g]_y", "A_y [g]_y", "A_z [g]_y"],
        "Magnetometer": ["M_x [gauss]", "M_y [gauss]", "M_z [gauss]"],
        "Gyroscope": ["G_x [mdps]", "G_y [mdps]", "G_z [mdps]"],
        "Temperature": ["TEMP"],
        "Pressure": ["PRESS"],
    }

    # If user requests all sensors, return every column
    if sensor_list == 'all':
        return [col for cols in available_sensors.values() for col in cols]

    selected_sensors = []
    # Iterate through requested sensor names
    for sensor in sensor_list:
        # Raise error if sensor name is invalid
        if sensor not in available_sensors:
            raise ValueError(
                f"Sensor '{sensor}' is not available. Choose from {list(available_sensors.keys())}"
            )
        # Append all columns for the valid sensor
        selected_sensors.extend(available_sensors[sensor])

    return selected_sensors

# Function to split data arrays into train and validation subsets
def split_train_validation(X, y_io, y_scene, stats, val_split=0.2, stratify_on=None):
    # Use stratified split if a column is provided for stratification
    if stratify_on is not None:
        X_train, X_val, y_io_train, y_io_val, y_scene_train, y_scene_val, stats_train, stats_val = train_test_split(
            X, y_io, y_scene, stats,
            test_size=val_split,
            random_state=42,
            stratify=stratify_on
        )
    else:
        # Simple contiguous split if no stratification
        split_idx = int((1 - val_split) * len(X))
        X_train, X_val = X[:split_idx], X[split_idx:]
        y_io_train, y_io_val = y_io[:split_idx], y_io[split_idx:]
        y_scene_train, y_scene_val = y_scene[:split_idx], y_scene[split_idx:]
        stats_train, stats_val = stats[:split_idx], stats[split_idx:]

    return (
        X_train, X_val,
        y_io_train, y_io_val,
        y_scene_train, y_scene_val,
        stats_train, stats_val
    )

def print_distribution(name, labels, encoder):
    # labels: 1D array of encoded ints
    unique_lbls, counts = np.unique(labels, return_counts=True)
    print(f"\n=== {name} Set ===")
    for enc, cnt in zip(unique_lbls, counts):
        human = encoder.inverse_transform([enc])[0]
        print(f"  {human:>10s}: {cnt}")

# Prepare sensor list and data preprocessor
sensors = get_sensors('all')  # Retrieve all sensor columns
preprocessor = DataPreprocessor(window_size=WINDOW_SIZE, target_samples=None)

# Load raw CSV data
train_data = pd.read_csv(TRAIN_CSV)
test_data = pd.read_csv(TEST_CSV)

# Preprocess data into feature windows, stats, and labels
X_train, stats_train, y_io_train, y_scene_train = preprocessor.preprocess_data(train_data, sensors)
X_test, stats_test, y_io_test, y_scene_test = preprocessor.preprocess_data(test_data, sensors)

# Print data shapes and unique classes for sanity
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Unique I/O classes: {np.unique(y_io_train)}")
print(f"Unique Scene classes: {np.unique(y_scene_train)}")

# Encode string labels to integers for I/O and Scene tasks
io_encoder = LabelEncoder()
scene_encoder = LabelEncoder()

y_io_train_encoded = io_encoder.fit_transform(y_io_train)
y_scene_train_encoded = scene_encoder.fit_transform(y_scene_train)
y_io_test_encoded = io_encoder.transform(y_io_test)
y_scene_test_encoded = scene_encoder.transform(y_scene_test)

# Number of classes for each task
num_io_classes = len(io_encoder.classes_)
num_scene_classes = len(scene_encoder.classes_)

print(f"Number of I/O classes: {num_io_classes}")
print(f"Number of Scene classes: {num_scene_classes}")

# Split data into training and validation sets
X_train_split, X_val, y_io_train_split, y_io_val, y_scene_train_split, y_scene_val, stats_train_split, stats_val = split_train_validation(
    X_train, y_io_train_encoded, y_scene_train_encoded,
    stats_train,
    val_split=0.2,
    stratify_on=y_scene_train_encoded  # Stratify by scene class distribution
)

# Decode validation labels for potential inspection or debugging
y_io_val_decoded = io_encoder.inverse_transform(y_io_val)
y_scene_val_decoded = scene_encoder.inverse_transform(y_scene_val)

# %% [code]
# Create PyTorch datasets for train, validation, and test splits
train_dataset = HARDSdataset(X_train_split, y_io_train_split, y_scene_train_split, io_encoder, scene_encoder)
val_dataset = HARDSdataset(X_val, y_io_val, y_scene_val, io_encoder, scene_encoder)
test_dataset = HARDSdataset(X_test, y_io_test_encoded, y_scene_test_encoded, io_encoder, scene_encoder)

# Build DataLoaders for batching during training and evaluation
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)
test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)

# Print dataset sizes for verification
print(f"Train samples: {len(train_loader.dataset)}")
print(f"Val samples:   {len(val_loader.dataset)}")
print(f"Test samples:  {len(test_loader.dataset)}")

# Print dataset labels distributions
print_distribution("Train I/O",   train_dataset.y_io,    train_dataset.io_label_encoder)
print_distribution("Train Scene", train_dataset.y_scene, train_dataset.scene_label_encoder)

print_distribution("Val I/O",     val_dataset.y_io,      val_dataset.io_label_encoder)
print_distribution("Val Scene",   val_dataset.y_scene,   val_dataset.scene_label_encoder)

print_distribution("Test I/O",    test_dataset.y_io,     test_dataset.io_label_encoder)
print_distribution("Test Scene",  test_dataset.y_scene,  test_dataset.scene_label_encoder)

Training data shape: (21476, 600, 11)
Test data shape: (5342, 600, 11)
Unique I/O classes: ['Indoor' 'Outdoor']
Unique Scene classes: ['Bar' 'Hallway' 'Office' 'Street']
Number of I/O classes: 2
Number of Scene classes: 4
Train samples: 17180
Val samples:   4296
Test samples:  5342

=== Train I/O Set ===
      Indoor: 12321
     Outdoor: 4859

=== Train Scene Set ===
         Bar: 4000
     Hallway: 2611
      Office: 5710
      Street: 4859

=== Val I/O Set ===
      Indoor: 3081
     Outdoor: 1215

=== Val Scene Set ===
         Bar: 1000
     Hallway: 653
      Office: 1428
      Street: 1215

=== Test I/O Set ===
      Indoor: 3809
     Outdoor: 1533

=== Test Scene Set ===
         Bar: 828
     Hallway: 1097
      Office: 1884
      Street: 1533


In [5]:
# Access the label encoders from your dataset object
io_encoder = train_loader.dataset.io_label_encoder
scene_encoder = train_loader.dataset.scene_label_encoder

# The .classes_ attribute holds the class names in order of their integer encoding (0, 1, 2, ...)
print("I/O Encoder Classes:", io_encoder.classes_)
print("Scene Encoder Classes:", scene_encoder.classes_)

# For a more robust way to use this in your code, create a mapping dictionary
io_map = {name: i for i, name in enumerate(io_encoder.classes_)}
scene_map = {name: i for i, name in enumerate(scene_encoder.classes_)}

print("\n--- Mappings ---")
print("I/O Mapping:", io_map)
print("Scene Mapping:", scene_map)

# Now you can directly use the names to get the encoding
print(f"\nEncoding for 'Indoor': {io_map['Indoor']}")
print(f"Encoding for 'Outdoor': {io_map['Outdoor']}")

I/O Encoder Classes: ['Indoor' 'Outdoor']
Scene Encoder Classes: ['Bar' 'Hallway' 'Office' 'Street']

--- Mappings ---
I/O Mapping: {'Indoor': 0, 'Outdoor': 1}
Scene Mapping: {'Bar': 0, 'Hallway': 1, 'Office': 2, 'Street': 3}

Encoding for 'Indoor': 0
Encoding for 'Outdoor': 1


# Augmented Dataset

In [45]:
# --- AUGMENTATION FUNCTIONS FOR IMU DATA (CORRECTED) ---

def gaussian_noise(sample, noise_std=0.05):
    """Adds Gaussian noise to the sample."""
    noise = np.random.normal(0, noise_std, sample.shape).astype(np.float32)
    return sample + noise

def scale(sample, scale_range=(0.9, 1.1)):
    """Scales the amplitude of the entire sample."""
    # CORRECTED LINE: Removed .astype() from the float
    scale_factor = np.random.uniform(scale_range[0], scale_range[1])
    return sample * scale_factor

def shift(sample, max_shift_points=30):
    """Randomly shifts the sample in time."""
    shift_points = np.random.randint(-max_shift_points, max_shift_points + 1)
    return np.roll(sample, shift_points, axis=0)

# --- THE NEW DATASET CLASS (No changes here, but included for completeness) ---

class AugmentedHARDSdataset(Dataset):
    def __init__(self, X, y_io, y_scene, io_label_encoder, scene_label_encoder, is_train=False):
        # Store data as NumPy arrays, not tensors
        self.X = X.astype(np.float32) 
        self.y_io = y_io
        self.y_scene = y_scene
        
        # Keep encoders for potential debugging
        self.io_label_encoder = io_label_encoder
        self.scene_label_encoder = scene_label_encoder
        
        # This flag is CRUCIAL. We only augment the training set.
        self.is_train = is_train
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        # Start with the original sample
        sample = self.X[idx]

        # --- Apply augmentations only if this is the training dataset ---
        if self.is_train:
            # Apply a sequence of augmentations.
            # You can add more or change the order.
            sample = gaussian_noise(sample, noise_std=0.05)
            sample = scale(sample, scale_range=(0.8, 1.2))
            sample = shift(sample, max_shift_points=int(sample.shape[0] * 0.1)) # Shift up to 10%
        
        # Convert the final sample to a tensor for the model
        # Labels also need to be converted to tensors here
        return (
            torch.from_numpy(sample),
            torch.as_tensor(self.y_io[idx], dtype=torch.long),
            torch.as_tensor(self.y_scene[idx], dtype=torch.long)
        )

In [43]:
# Function to map high-level sensor names to their corresponding DataFrame columns
def get_sensors(sensor_list):
    # Dictionary of available sensor groups and their DataFrame columns
    available_sensors = {
        "Accelerometer": ["A_x [g]_y", "A_y [g]_y", "A_z [g]_y"],
        "Magnetometer": ["M_x [gauss]", "M_y [gauss]", "M_z [gauss]"],
        "Gyroscope": ["G_x [mdps]", "G_y [mdps]", "G_z [mdps]"],
        "Temperature": ["TEMP"],
        "Pressure": ["PRESS"],
    }

    # If user requests all sensors, return every column
    if sensor_list == 'all':
        return [col for cols in available_sensors.values() for col in cols]

    selected_sensors = []
    # Iterate through requested sensor names
    for sensor in sensor_list:
        # Raise error if sensor name is invalid
        if sensor not in available_sensors:
            raise ValueError(
                f"Sensor '{sensor}' is not available. Choose from {list(available_sensors.keys())}"
            )
        # Append all columns for the valid sensor
        selected_sensors.extend(available_sensors[sensor])

    return selected_sensors

# Function to split data arrays into train and validation subsets
def split_train_validation(X, y_io, y_scene, stats, val_split=0.2, stratify_on=None):
    # Use stratified split if a column is provided for stratification
    if stratify_on is not None:
        X_train, X_val, y_io_train, y_io_val, y_scene_train, y_scene_val, stats_train, stats_val = train_test_split(
            X, y_io, y_scene, stats,
            test_size=val_split,
            random_state=42,
            stratify=stratify_on
        )
    else:
        # Simple contiguous split if no stratification
        split_idx = int((1 - val_split) * len(X))
        X_train, X_val = X[:split_idx], X[split_idx:]
        y_io_train, y_io_val = y_io[:split_idx], y_io[split_idx:]
        y_scene_train, y_scene_val = y_scene[:split_idx], y_scene[split_idx:]
        stats_train, stats_val = stats[:split_idx], stats[split_idx:]

    return (
        X_train, X_val,
        y_io_train, y_io_val,
        y_scene_train, y_scene_val,
        stats_train, stats_val
    )

def print_distribution(name, labels, encoder):
    # labels: 1D array of encoded ints
    unique_lbls, counts = np.unique(labels, return_counts=True)
    print(f"\n=== {name} Set ===")
    for enc, cnt in zip(unique_lbls, counts):
        human = encoder.inverse_transform([enc])[0]
        print(f"  {human:>10s}: {cnt}")

# Prepare sensor list and data preprocessor
sensors = get_sensors('all')  # Retrieve all sensor columns
preprocessor = DataPreprocessor(window_size=WINDOW_SIZE, target_samples=None)

# Load raw CSV data
train_data = pd.read_csv(TRAIN_CSV)
test_data = pd.read_csv(TEST_CSV)

# Preprocess data into feature windows, stats, and labels
X_train, stats_train, y_io_train, y_scene_train = preprocessor.preprocess_data(train_data, sensors)
X_test, stats_test, y_io_test, y_scene_test = preprocessor.preprocess_data(test_data, sensors)

# Print data shapes and unique classes for sanity
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Unique I/O classes: {np.unique(y_io_train)}")
print(f"Unique Scene classes: {np.unique(y_scene_train)}")

# Encode string labels to integers for I/O and Scene tasks
io_encoder = LabelEncoder()
scene_encoder = LabelEncoder()

y_io_train_encoded = io_encoder.fit_transform(y_io_train)
y_scene_train_encoded = scene_encoder.fit_transform(y_scene_train)
y_io_test_encoded = io_encoder.transform(y_io_test)
y_scene_test_encoded = scene_encoder.transform(y_scene_test)

# Number of classes for each task
num_io_classes = len(io_encoder.classes_)
num_scene_classes = len(scene_encoder.classes_)

print(f"Number of I/O classes: {num_io_classes}")
print(f"Number of Scene classes: {num_scene_classes}")

# Split data into training and validation sets
X_train_split, X_val, y_io_train_split, y_io_val, y_scene_train_split, y_scene_val, stats_train_split, stats_val = split_train_validation(
    X_train, y_io_train_encoded, y_scene_train_encoded,
    stats_train,
    val_split=0.2,
    stratify_on=y_scene_train_encoded  # Stratify by scene class distribution
)

# Decode validation labels for potential inspection or debugging
y_io_val_decoded = io_encoder.inverse_transform(y_io_val)
y_scene_val_decoded = scene_encoder.inverse_transform(y_scene_val)

# %% [code]
# Create PyTorch datasets for train, validation, and test splits
train_dataset = AugmentedHARDSdataset(X_train_split, y_io_train_split, y_scene_train_split, io_encoder, scene_encoder, is_train=True)
val_dataset = AugmentedHARDSdataset(X_val, y_io_val, y_scene_val, io_encoder, scene_encoder, is_train=False)
test_dataset = AugmentedHARDSdataset(X_test, y_io_test_encoded, y_scene_test_encoded, io_encoder, scene_encoder, is_train=False)

# Build DataLoaders for batching during training and evaluation
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)
test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)

# Print dataset sizes for verification
print(f"Train samples: {len(train_loader.dataset)}")
print(f"Val samples:   {len(val_loader.dataset)}")
print(f"Test samples:  {len(test_loader.dataset)}")

# Print dataset labels distributions
print_distribution("Train I/O",   train_dataset.y_io,    train_dataset.io_label_encoder)
print_distribution("Train Scene", train_dataset.y_scene, train_dataset.scene_label_encoder)

print_distribution("Val I/O",     val_dataset.y_io,      val_dataset.io_label_encoder)
print_distribution("Val Scene",   val_dataset.y_scene,   val_dataset.scene_label_encoder)

print_distribution("Test I/O",    test_dataset.y_io,     test_dataset.io_label_encoder)
print_distribution("Test Scene",  test_dataset.y_scene,  test_dataset.scene_label_encoder)

Training data shape: (21476, 600, 11)
Test data shape: (5342, 600, 11)
Unique I/O classes: ['Indoor' 'Outdoor']
Unique Scene classes: ['Bar' 'Hallway' 'Office' 'Street']
Number of I/O classes: 2
Number of Scene classes: 4
Train samples: 17180
Val samples:   4296
Test samples:  5342

=== Train I/O Set ===
      Indoor: 12321
     Outdoor: 4859

=== Train Scene Set ===
         Bar: 4000
     Hallway: 2611
      Office: 5710
      Street: 4859

=== Val I/O Set ===
      Indoor: 3081
     Outdoor: 1215

=== Val Scene Set ===
         Bar: 1000
     Hallway: 653
      Office: 1428
      Street: 1215

=== Test I/O Set ===
      Indoor: 3809
     Outdoor: 1533

=== Test Scene Set ===
         Bar: 828
     Hallway: 1097
      Office: 1884
      Street: 1533


# Depthwise Separable Convolution and Early-Exit CNN Module
This cell defines a depthwise-separable 1D convolution helper and the main Early-Exit CNN model with dynamic confidence-based exits and battery-aware thresholds.

In [5]:
def allowed_depth(battery_level):
    b = float(battery_level.min().item()) if torch.is_tensor(battery_level) else float(battery_level)
    return 1 if b < 0.25 else 2

def map_battery(b, k=10.0, center=0.5):
    """
    Map battery_level to sensitivity curve via sigmoid centered at 0.5.
    More sensitive around mid battery range.
    """
    return torch.sigmoid(k*(b-center))
    
def tau_piecewise(b, offset: float = 0.0):
    """
    Battery → confidence threshold τ.
    Use `offset` (e.g., -0.003) to slightly increase early-exit rates globally.
    """
    # Helper to handle tensor/float uniformly
    def _tensor(v, like):
        return torch.tensor(v, device=like.device, dtype=like.dtype) if torch.is_tensor(like) else v

    if torch.is_tensor(b):
        tau = torch.full_like(b, 0.972 + offset)   # ≥ 0.80
        tau = torch.where(b < 0.80, _tensor(0.952 + offset, b), tau)  # 0.60–0.80
        tau = torch.where(b < 0.60, _tensor(0.920 + offset, b), tau)  # 0.40–0.60 (unchanged)
        tau = torch.where(b < 0.40, _tensor(0.850 + offset, b), tau)  # 0.25–0.40
        tau = torch.where(b < 0.25, _tensor(0.750 + offset, b), tau)  # < 0.25
        return tau.clamp_(0.50, 0.999)
    else:
        # scalar path
        if b < 0.25:  tau = 0.750
        elif b < 0.40: tau = 0.850
        elif b < 0.60: tau = 0.920
        elif b < 0.80: tau = 0.952
        else:          tau = 0.972
        tau += offset
        return float(min(max(tau, 0.50), 0.999))

In [33]:
# ------------------------------
# Depthwise separable 1D conv
# ------------------------------
def depthwise_separable_conv1d(in_ch, out_ch, kernel_size, stride=1, padding=0, drop_p=0.2, dilation=1):
    """
    DW (groups=in_ch) -> BN -> ReLU -> PW(1x1) -> BN -> ReLU -> Dropout1d
    """
    return nn.Sequential(
        nn.Conv1d(in_ch, in_ch, kernel_size=kernel_size, stride=stride,
                  padding=padding, dilation=dilation, groups=in_ch, bias=False),
        nn.BatchNorm1d(in_ch),
        nn.ReLU(inplace=True),
        nn.Conv1d(in_ch, out_ch, kernel_size=1, bias=False),
        nn.BatchNorm1d(out_ch),
        nn.ReLU(inplace=True),
        nn.Dropout1d(drop_p)
    )

class DepthwiseEarlyExitCNN1D(nn.Module):
    def __init__(self, if_train, train_shape, n_io, n_scene):
        super().__init__()
        self.if_train = if_train
        seq_len, n_sensors = train_shape

        # Backbone
        # self.backbone = nn.ModuleList([
        #     nn.Sequential(
        #         depthwise_separable_conv1d(n_sensors, 16, kernel_size=6, stride=3, padding=2, drop_p=0.1),
        #         depthwise_separable_conv1d(16, 32, kernel_size=5, stride=2, padding=2, drop_p=0.1)
        #     ),
        #     depthwise_separable_conv1d(32, 64, kernel_size=5, stride=3, padding=1, drop_p=0.2)
        # ])
        self.backbone0 = nn.Sequential(
            depthwise_separable_conv1d(n_sensors, 16, kernel_size=6, stride=3, padding=2, drop_p=0.1),
            depthwise_separable_conv1d(16, 32, kernel_size=5, stride=2, padding=2, drop_p=0.1)
        )

        # ---- Task-Specific Branches ----
        # Branch 1: Deep I/O refinement
        self.io_branch = depthwise_separable_conv1d(32, 64, kernel_size=6, stride=3, padding=2, drop_p=0.2)
        # Branch 2: Scene classification
        self.scene_branch = depthwise_separable_conv1d(32, 48, kernel_size=3, stride=2, padding=1, drop_p=0.3)

        # Exit heads
        self.io_head0 = nn.Sequential(nn.AdaptiveAvgPool1d(1), nn.Flatten(), nn.Linear(32, n_io))
        self.io_head1 = nn.Sequential(nn.AdaptiveAvgPool1d(1), nn.Flatten(), nn.Linear(32 + 64, n_io))
        self.scene_head1 = nn.Sequential(nn.AdaptiveAvgPool1d(1), nn.Flatten(), nn.Linear(48, n_scene))
        # self.io_head1 = MultiLevelIOHead(c0=32, c1=64, n_io=n_io)

        # Adapter (kept) to project E0 logits into Scene space if you fuse later
        self.ensemble_adapter = nn.Linear(n_io, n_scene)

    # max softmax confidence per sample
    def get_confidence(self, logits):
        return F.softmax(logits, dim=-1).max(dim=-1)[0]

    def forward(self, x, battery_level=1.0):
        # Input: [B, T, C_in] -> [B, C_in, T]
        x = x.permute(0, 2, 1)

        # -------- Stage 1 (E0: I/O) --------
        x0 = self.backbone0(x) # [B, 32, T/4]
        e0io = self.io_head0(x0) # logits I/O @ E0
        conf0 = self.get_confidence(e0io)

        # Compute tau (external function you already have)
        tau = tau_piecewise(battery_level if not torch.is_tensor(battery_level) else float(battery_level.min().item()))

        # -------- Stage 2 (Exit-2 features) --------
        # x1 = self.backbone0(x0) # [B, last_c, T/8]
        x1_io = self.io_branch(x0)      # Features specialized for I/O
        x1_scene = self.scene_branch(x0)  # Features specialized for Scene

        # ---- Heads use their specialized features ----
        e1sc = self.scene_head1(x1_scene)

        # e1sc = self.scene_head1(x1) # logits Scene @ E1
        # e1io = self.io_head1(x1) # logits I/O deep @ E1
        x0_resized = F.adaptive_avg_pool1d(x0, x1_io.shape[-1])
        x_fused_io = torch.cat([x0_resized, x1_io], dim=1) # [B, 96, T/8]
        e1io = self.io_head1(x_fused_io)
        # conf1 = self.get_confidence(e1io)

        # ---------------- TRAINING ----------------
        if self.if_train:
            should_exit0 = (conf0 >= tau_piecewise(battery_level)).float()
            p_exits = torch.stack([should_exit0, 1.0 - should_exit0], dim=1)
            return [e0io, e1sc, e1io], p_exits

        # ---------------- INFERENCE ----------------
        if allowed_depth(battery_level) == 1:
            return [e0io], 1

        if float(conf0[0].item()) >= float(tau if not torch.is_tensor(tau) else tau.min().item()):
            return [e0io], 1
        else:
            # your existing policy: take deep I/O as the final I/O
            io_final = e1io
            return [io_final, e1sc], 2

# Training and Evaluation Loop
The following function runs one epoch over the dataset, computing classification loss, expected computational cost loss, and updating model parameters when in training mode.

In [7]:
def compute_class_weights(class_counts, method='inverse_freq'):
    """
    Compute class weights for handling imbalance.
    
    Args:
        class_counts (list): Number of samples per class [Bar, Hallway, Office, Street]
        method (str): 'inverse_freq' or 'balanced'
    
    Returns:
        torch.Tensor: Weights for each class
    """
    counts = np.array(class_counts, dtype=np.float32)
    
    if method == 'inverse_freq':
        # Standard inverse frequency: weight = total_samples / (n_classes * class_count)
        total = counts.sum()
        weights = total / (len(counts) * counts)
    
    elif method == 'balanced':
        # Sklearn-style balanced weights: weight = total_samples / (n_classes * class_count)
        # Same as inverse_freq but explicit naming
        total = counts.sum()
        weights = total / (len(counts) * counts)
    
    return torch.tensor(weights, dtype=torch.float32)

# Your training scene counts: [Bar, Hallway, Office, Street]
train_scene_counts = [4000, 2611, 5710, 4859]

# Compute class weights
scene_weights = compute_class_weights(train_scene_counts, method='inverse_freq')
print(f"Class weights: {scene_weights}")

class WeightedSceneLoss(nn.Module):
    """
    Weighted Cross-Entropy Loss for scene classification with class imbalance handling.
    """
    def __init__(self, class_weights, device='cpu', label_smoothing=0.0):
        super().__init__()
        self.class_weights = class_weights.to(device)
        self.label_smoothing = label_smoothing
        
    def forward(self, logits, targets):
        """
        Args:
            logits: [batch_size, n_classes] scene predictions
            targets: [batch_size] class indices (0=Bar, 1=Hallway, 2=Office, 3=Street)
        """
        # Use PyTorch's built-in weighted CrossEntropyLoss
        loss_fn = nn.CrossEntropyLoss(
            weight=self.class_weights, 
            label_smoothing=self.label_smoothing
        )
        return loss_fn(logits, targets)

# Initialize the weighted loss
weighted_scene_loss = WeightedSceneLoss(
    class_weights=scene_weights, 
    device=device,
    label_smoothing=0.05  # Optional: adds regularization
)

Class weights: tensor([1.0738, 1.6450, 0.7522, 0.8839])


In [27]:
def _top1_correct(logits: torch.Tensor, labels: torch.Tensor):
    """Returns (#correct, #samples) for top-1 accuracy."""
    return (logits.argmax(dim=1) == labels).sum().item(), labels.size(0)

def run_epoch(model, loader, criterion, optimizer, scheduler, train, lambda_cost=0.05, exit_costs=(0.4, 1.0), battery_sampler=None):
    # We want p_exits from the model even in eval, so keep if_train=True
    model.if_train = True
    model.train() if train else model.eval()

    # MODIFIED: Expanded stats dict to track all heads individually
    stats = {
        "loss_total": 0.0,
        "n":          0,
        "correct_io0": 0,  # Correct predictions for I/O at Exit 0
        "correct_sc1": 0,  # Correct predictions for Scene at Exit 1
        "correct_io1": 0,  # Correct predictions for I/O at Exit 1
    }

    with torch.set_grad_enabled(train):
        for feats, io_lbls, sc_lbls in loader:
            feats   = feats.float().to(device)       # [B,T,C]
            io_lbls = io_lbls.view(-1).to(device)    # [B]
            sc_lbls = sc_lbls.view(-1).to(device)    # [B]
            B = feats.size(0)

            batt = (battery_sampler(B).to(device) if battery_sampler else torch.ones(B, device=device))

            # Model returns: (e0io, e1sc, e1io), p_exits with p_exits shape [B,2]
            (e0io, e1sc, e1io), p_exits = model(feats, battery_level=batt)

            # --- losses ---
            loss_e0io = criterion(e0io, io_lbls)         # I/O @ exit-1
            loss_e1sc = weighted_scene_loss(e1sc, sc_lbls) # Scene @ exit-2

            # I/O deep (only care about late subset): weight by p(exit=2)
            w_late = p_exits[:, 1].detach()              # [B]
            loss_e1io_per = F.cross_entropy(e1io, io_lbls, reduction='none')  # [B]
            loss_e1io = (w_late * loss_e1io_per).mean()

            # Classification loss mix
            w0, w1, w0d = 3.0, 10.0, 10.0
            cls_loss = w0*loss_e0io + w1*loss_e1sc + w0d*loss_e1io
            
            loss = cls_loss

            if train:
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                # if scheduler is per-step, uncomment:
                # if scheduler is not None: scheduler.step()

            # --- stats ---
            stats["loss_total"] += loss.item() * B
            stats["n"] += B

            # MODIFIED: Calculate accuracy for ALL heads on ALL samples.
            # This allows you to see the raw performance of each head before the exit policy is applied.
            c, _ = _top1_correct(e0io, io_lbls)
            stats["correct_io0"] += c
            
            c, _ = _top1_correct(e1sc, sc_lbls)
            stats["correct_sc1"] += c

            # This is the crucial new metric you need to watch.
            c, _ = _top1_correct(e1io, io_lbls)
            stats["correct_io1"] += c

    n = stats["n"]
    losses = {
        "total":   stats["loss_total"] / n
    }
    
    # MODIFIED: Renamed and expanded the accuracies dictionary for clarity.
    accs = {
        "io@0_early": stats["correct_io0"] / n,
        "sc@1_deep":  stats["correct_sc1"] / n,
        "io@1_deep":  stats["correct_io1"] / n, # The accuracy of the second I/O head
    }
    return losses, accs

In [9]:
def battery_sampler_uniform(batch_size):
    """
    Returns a tensor of shape (batch_size,)
    with battery levels sampled uniformly in [0, 1].
    """
    return torch.rand(batch_size)

def battery_sampler_beta(batch_size, alpha=2.0, beta=5.0):
    """
    Returns a tensor of shape (batch_size,)
    with battery levels sampled from a Beta distribution.
    
    Parameters:
    - alpha, beta: shape parameters of the Beta distribution.
      A Beta(2,5) mimics a device which spends more time at lower charge.
    """
    return torch.distributions.Beta(alpha, beta).sample((batch_size,))

def battery_sampler_schedule(batch_size, start=1.0, end=0.0, epoch=0, total_epochs=100):
    """
    Returns a tensor where battery levels linearly decay over training epochs.
    Useful for curriculum learning.
    
    Parameters:
    - start: initial battery level at epoch=0
    - end: final battery level at epoch=total_epochs
    - epoch: current epoch index
    - total_epochs: total number of epochs
    """
    level = start + (end - start) * (epoch / (total_epochs - 1))
    return torch.full((batch_size,), level)

# def battery_sampler_schedule(bs, epoch, total_epochs):
#     # A rapid drop after 70 % (Li-ion plateau), then flat, then tail
#     pct = epoch / total_epochs
#     level = 1.0 - (0.9*pct**0.5 + 0.1*pct)      # convex curve
#     noise  = torch.randn(bs)*0.05               # small diversity
#     return torch.clamp(torch.full((bs,), level) + noise, 0, 1)

In [22]:
def train_model(train_loader, val_loader, hparams, save_dir, run_name):
    run_save_dir = Path(save_dir) / run_name
    run_save_dir.mkdir(parents=True, exist_ok=True)

    # ── model ----------------------------------------------------------------
    sample_shape = train_loader.dataset[0][0].shape
    model = DepthwiseEarlyExitCNN1D(
        True,
        sample_shape,
        len(train_loader.dataset.io_label_encoder.classes_),
        len(train_loader.dataset.scene_label_encoder.classes_)
    ).to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.AdamW(model.parameters(), lr=hparams["lr"], weight_decay=hparams["weight_decay"])

    # ── scheduler ------------------------------------------------------------
    if hparams["scheduler"] == "step":
        scheduler = StepLR(optimizer, step_size=hparams["step_size"], gamma=0.5)
    elif hparams["scheduler"] == "plateau":
        scheduler = ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=5, min_lr=1e-7)
    elif hparams["scheduler"] == "onecycle":
        total_steps = hparams["epochs"] * len(train_loader)
        scheduler = OneCycleLR(optimizer, max_lr=1e-2, total_steps=total_steps, pct_start=hparams["pc"], div_factor=hparams["divfactor"], final_div_factor=1e4)
    else:
        raise ValueError(f"Unsupported scheduler: {hparams['scheduler']}")

    # ── early-stopping config -----------------------------------------------
    monitor = "total"
    higher_is_better = False               # flip if you monitor a loss
    min_delta = 1e-4
    patience = 7

    best_value = -float("inf") if higher_is_better else float("inf")
    epochs_no_improve = 0
    history = []

    # ── epoch loop -----------------------------------------------------------
    for epoch in range(1, hparams["epochs"] + 1):
        # ---- train ----------------------------------------------------------
        tr_losses, tr_accs = run_epoch(
            model, train_loader, criterion, optimizer, scheduler,
            train=True,
            lambda_cost=hparams["lambda_cost"],
            battery_sampler=lambda bs: battery_sampler_schedule(bs, epoch=epoch, total_epochs=hparams["epochs"])
        )

        # ---- validation -----------------------------------------------------
        vl_losses, vl_accs = run_epoch(
            model, val_loader, criterion, None, None,
            train=False,
            lambda_cost= hparams["lambda_cost"],
            battery_sampler=lambda bs: torch.ones(bs, device=device)
        )

        # ---- scheduler step -------------------------------------------------
        if hparams["scheduler"] == "step":
            scheduler.step()
        elif hparams["scheduler"] == "plateau":
            scheduler.step(vl_losses[monitor])
        elif hparams["scheduler"] == "onecycle":
            scheduler.step()

        # ---- pretty log -----------------------------------------------------
        print(
            f"Ep {epoch:03d} | "
            f"tr L:{tr_losses['total']:.3f} "
            f"IO_0[{tr_accs['io@0_early']:.2%}] "
            f"IO_1[{tr_accs['io@1_deep']:.2%}] "
            f"SC_1[{tr_accs['sc@1_deep']:.2%}] | "
            f"vl L:{vl_losses['total']:.3f} "
            f"IO_0[{vl_accs['io@0_early']:.2%}] "
            f"IO_1[{vl_accs['io@1_deep']:.2%}] "
            f"SC_1[{vl_accs['sc@1_deep']:.2%}] "
        )

        # ---- early-stopping -------------------------------------------------
        curr_value = vl_losses[monitor]
        improved = (curr_value > best_value + min_delta) if higher_is_better else (curr_value < best_value - min_delta)

        if improved:
            best_value = curr_value
            epochs_no_improve = 0
            torch.save(model.state_dict(), run_save_dir / "best_model.pth")
            print(f" -> Saved best model ({monitor}: {best_value:.4f})")
        else:
            epochs_no_improve += 1
            print(f" -> No improvement on {monitor} ({epochs_no_improve}/{patience})")

        if epochs_no_improve >= patience:
            print(f" -> Early stopping at epoch {epoch}")
            break

        # ---- bookkeeping ----------------------------------------------------
        history.append({
            "epoch" : epoch,
            **{f"tr_{k}": v for k, v in tr_losses.items()},
            **{f"vl_{k}": v for k, v in vl_losses.items()}
        })

    print(f"END RUN: {run_name} | best {monitor}: {best_value:.4f}")
    
    return history

In [48]:
# List of hyperparameter combinations to explore
hyperparameter_space = [
    # {'lr': 1e-3, 'weight_decay': 1e-4, 'scheduler': 'onecycle', 'epochs': 50, 'lambda_cost': 0.0, 'drop_p': 0.0, 'pc': 0.15, 'divfactor': 18.0, 'x': 5},
    # {'lr': 1e-3, 'weight_decay': 1e-4, 'scheduler': 'onecycle', 'epochs': 50, 'lambda_cost': 0.05, 'drop_p': 0.0, 'pc': 0.70, 'divfactor': 18.0, 'x': 9},
    # {'lr': 3e-4, 'weight_decay': 1e-5, 'scheduler': 'step', 'step_size': 20, 'epochs': 30, 'lambda_cost': 0.05, 'drop_p': 0.0, 'x': 64}, # BEST EVER !!!!!!!! # 27 kB
    # {'lr': 3e-4, 'weight_decay': 1e-5, 'scheduler': 'step', 'step_size': 20, 'epochs': 30, 'lambda_cost': 0.05, 'drop_p': 0.0, 'x': 15}, # BEST EVER !!!!!!!! # 52Kb
    # {'lr': 3e-4, 'weight_decay': 1e-5, 'scheduler': 'step', 'step_size': 20, 'epochs': 30, 'lambda_cost': 0.05, 'drop_p': 0.0, 'x': 90},
    # {'lr': 3e-4, 'weight_decay': 5e-5, 'scheduler': 'step', 'step_size': 20, 'epochs': 30, 'lambda_cost': 0.05, 'drop_p': 0.0, 'x': 91}, # ooo
    {'lr': 3e-4, 'weight_decay': 5e-5, 'scheduler': 'step', 'step_size': 20, 'epochs': 50, 'lambda_cost': 0.05, 'x': 113}, # WOWW
    # {'lr': 5e-4, 'weight_decay': 1e-5, 'scheduler': 'step', 'step_size': 10, 'epochs': 50, 'lambda_cost': 0.0, 'drop_p': 0.0, 'x': 4}, # BEST
    # {'lr': 7e-4, 'weight_decay': 1e-5, 'scheduler': 'step', 'step_size': 20, 'epochs': 50, 'lambda_cost': 0.0, 'drop_p': 0.0, 'x': 4}, # BEST

    # {'lr': 2e-2, 'weight_decay': 5e-4, 'scheduler': 'step', 'step_size': 20, 'epochs': 50, 'lambda_cost': 0.0, 'drop_p': 0.0, 'x': 1}, # HIGH ACC BUT BAD CURVE
    # {'lr': 2e-2, 'weight_decay': 5e-4, 'scheduler': 'step', 'step_size': 20, 'epochs': 50, 'lambda_cost': 0.05, 'drop_p': 0.0, 'x': 2},
    # {'lr': 2e-2, 'weight_decay': 5e-4, 'scheduler': 'step', 'step_size': 20, 'epochs': 50, 'lambda_cost': 0.1, 'drop_p': 0.0, 'x': 3}, 
    # {'lr': 1e-2, 'weight_decay': 5e-4, 'scheduler': 'step', 'step_size': 20, 'epochs': 50, 'lambda_cost': 0.0, 'drop_p': 0.0, 'x': 40}, # QUITE GOOD
    # {'lr': 1e-3, 'weight_decay': 1e-4, 'scheduler': 'onecycle', 'epochs': 50, 'lambda_cost': 0.0, 'drop_p': 0.0, 'pc': 0.10, 'divfactor': 25.0, 'x': 20}, # il migliore
    # {'lr': 5e-3, 'weight_decay': 5e-4, 'scheduler': 'onecycle', 'epochs': 50, 'lambda_cost': 0.0, 'drop_p': 0.0, 'pc': 0.10, 'divfactor': 30.0, 'x': 20}, # il migliore
    # {'lr': 5e-3, 'weight_decay': 1e-4, 'scheduler': 'step', 'step_size': 20, 'epochs': 20, 'lambda_cost': 0.0, 'drop_p': 0.0, 'x': 40}, # QUITE GOOD
]

# Directory to save checkpoint runs
SAVE_DIR = "/kaggle/working/validation_runs"

# Dictionary to collect training histories
all_histories = {}
for hparams in hyperparameter_space:
    # Construct a short run name for saving and lookup
    run_name = f"LR_{hparams['lr']}_WD_{hparams['weight_decay']}_SCH_{hparams['scheduler']}_X{hparams['x']}"
    print(f'\nTraining: {run_name}')
    # Train the model with the given hyperparameters
    history = train_model(
        train_loader=train_loader,
        val_loader=val_loader,
        hparams=hparams,
        save_dir=SAVE_DIR,
        run_name=run_name
    )
    # Store history for analysis and comparison
    all_histories[run_name] = history

print("Hyperparameter tuning completed.")


Training: LR_0.0003_WD_5e-05_SCH_step_X113
Ep 001 | tr L:19.301 IO_0[70.02%] IO_1[74.70%] SC_1[41.86%] vl L:16.167 IO_0[78.61%] IO_1[83.66%] SC_1[62.69%] 
 -> Saved best model (total: 16.1669)
Ep 002 | tr L:16.205 IO_0[77.92%] IO_1[80.46%] SC_1[58.68%] vl L:14.374 IO_0[83.29%] IO_1[84.59%] SC_1[66.53%] 
 -> Saved best model (total: 14.3743)
Ep 003 | tr L:15.311 IO_0[80.91%] IO_1[82.17%] SC_1[61.36%] vl L:13.820 IO_0[84.31%] IO_1[85.57%] SC_1[66.74%] 
 -> Saved best model (total: 13.8198)
Ep 004 | tr L:14.856 IO_0[82.03%] IO_1[83.05%] SC_1[62.13%] vl L:13.278 IO_0[84.99%] IO_1[85.75%] SC_1[67.36%] 
 -> Saved best model (total: 13.2779)
Ep 005 | tr L:14.548 IO_0[82.43%] IO_1[83.36%] SC_1[63.10%] vl L:12.864 IO_0[85.50%] IO_1[86.31%] SC_1[68.18%] 
 -> Saved best model (total: 12.8640)
Ep 006 | tr L:14.265 IO_0[83.12%] IO_1[83.96%] SC_1[63.92%] vl L:12.802 IO_0[85.41%] IO_1[86.08%] SC_1[68.23%] 
 -> Saved best model (total: 12.8017)
Ep 007 | tr L:14.074 IO_0[83.27%] IO_1[84.10%] SC_1[64.7

/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  

In [49]:
plot_data = []
for run_name, history in all_histories.items():
    for ep in history:
        ep["run_name"] = run_name
        plot_data.append(ep)

df = pd.DataFrame(plot_data)

df["tr_scene_avg"] = df["tr_total"]
df["vl_scene_avg"] = df["vl_total"]

run_names = df["run_name"].unique()
n_runs = len(run_names)

# ------------------------------------ #
# 1.  LOSS curves (classification)      #
# ------------------------------------ #
plt.style.use("seaborn-v0_8-whitegrid")

# Define custom colors for training and validation lines
train_color = "blue"  # Color for training loss lines
val_color = "green"    # Color for validation loss lines

fig, ax1 = plt.subplots(figsize=(15, 8))

# Plot training loss with a specific color
sns.lineplot(
    data=df,
    x="epoch",
    y="tr_total",
    hue="run_name",
    style="run_name",
    dashes=True,
    palette={run: train_color for run in df["run_name"].unique()},  # Assign single color to all runs
    ax=ax1,
    legend=False,
    markersize=7
)

# Plot validation loss with a different color
sns.lineplot(
    data=df,
    x="epoch",
    y="vl_total",
    hue="run_name",
    style="run_name",
    dashes={r: (2, 2) for r in run_names},  # Dashed for validation
    palette={run: val_color for run in df["run_name"].unique()},  # Assign single color to all runs
    ax=ax1,
    legend=False
)

ax1.set(
    title="Classification-loss trend per run",
    xlabel="Epoch",
    ylabel="Loss"
)
ax1.legend(title="run", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout(rect=[0, 0, 0.82, 1])
plt.show()

In [50]:
@torch.no_grad()
def evaluate_model_performance(model, loader, battery_level: float):
    """
    Evaluates:
      - exit rates
      - I/O accuracies: io@1_early, io@2_final, io@late_subset_e0, io@1_overall
      - Scene accuracy at exit2 (after Outdoor->Street post-hoc)
      - Weighted F1s for I/O (all, late-final, late-E0) and Scene (raw/final @ exit2, and 'all' = late only)

    Assumptions:
      - model(if_train=True) returns outs = [e0io_logits, e1sc_logits, e1io_logits]
      - model is already on the correct device (we infer it)
      - global functions tau_piecewise(batt) and allowed_depth(batt) exist; if not, we fallback to depth-allowed
    """
    device = next(model.parameters()).device
    model.eval()

    total_samples   = 0
    exit_counts     = {1: 0, 2: 0}
    correct_at_exit = {1: 0, 2: 0}    # io@1 on early; scene@2 on late
    correct_io_late_final = 0         # final I/O on late subset (E2)
    correct_io_late_e0    = 0         # E0 "what-if" I/O on late subset
    correct_io_overall_e0 = 0         # E0 "force-all"

    # I/O F1 buffers
    y_true_io_all, y_pred_io_final_all = [], []                  # final I/O (E0 early, E1 I/O late)
    y_true_io_at_exit2, y_pred_io_at_exit2_final = [], []        # final I/O only on late
    y_true_io_at_exit2_e0,  y_pred_io_at_exit2_e0  = [], []      # E0 what-if on same late subset

    # Scene F1 buffers
    y_true_scene_all, y_pred_scene_all = [], []                  # effectively late-only
    y_true_scene_at_exit2_raw,  y_pred_scene_at_exit2_raw  = [], []
    y_true_scene_at_exit2_final, y_pred_scene_at_exit2_final = [], []

    # Label ids (robust to case)
    def _find_label_id(enc, name):
        names = list(enc.classes_)
        try:
            return names.index(name)
        except ValueError:
            # try case-insensitive
            lower = [n.lower() for n in names]
            return lower.index(name.lower())
    scene_encoder = loader.dataset.scene_label_encoder
    io_encoder    = loader.dataset.io_label_encoder
    street_label_int  = _find_label_id(scene_encoder, 'Street')
    outdoor_label_int = _find_label_id(io_encoder, 'Outdoor')

    # Precompute depth permission for this battery level (fallback to allowed)
    depth_allowed_global = True
    try:
        depth_allowed_global = (allowed_depth(battery_level) != 1)
    except Exception:
        pass  # assume depth allowed

    for feats, io_lbls, sc_lbls in loader:
        feats   = feats.to(device, non_blocking=True).float()  # [B,T,C]
        io_lbls = io_lbls.view(-1).to(device).long()           # [B]
        sc_lbls = sc_lbls.view(-1).to(device).long()           # [B]
        B = feats.size(0)
        total_samples += B

        # ---- One forward in "train-return" mode to get BOTH exits' logits ----
        was_flag = getattr(model, "if_train", False)
        model.if_train = True
        # batt vec just to satisfy signature; gating is done locally below
        batt_vec = torch.full((B,), float(battery_level), device=device)
        outs, _ = model(feats, battery_level=batt_vec)         # expect [e0io, e1sc, e1io]
        model.if_train = was_flag

        if len(outs) < 2:
            raise RuntimeError("Model(if_train=True) must return at least [e0io, e1sc].")

        e0_logits_io = outs[0]                # [B, n_io]
        e1_logits_sc = outs[1]                # [B, n_scene]
        e1_logits_io = outs[2] if len(outs) > 2 else None  # [B, n_io] (deep I/O), optional

        # ---- Per-sample routing (policy) computed LOCALLY ----
        conf0 = F.softmax(e0_logits_io, dim=-1).amax(dim=-1)  # [B]
        
        # tau may be a float or tensor; normalize to float
        try:
            tau_val = tau_piecewise(battery_level)
            tau_val = float(tau_val.item()) if hasattr(tau_val, "item") else float(tau_val)
        except Exception:
            tau_val = 0.9  # safe default if tau_piecewise is not available

        exit_mask2 = (conf0 < tau_val) & depth_allowed_global   # go deep
        exit_mask1 = ~exit_mask2                                # exit early

        exit_counts[1] += int(exit_mask1.sum().item())
        exit_counts[2] += int(exit_mask2.sum().item())

        # ---- Predictions ----
        e0_pred = e0_logits_io.argmax(1)                        # [B]
        if e1_logits_io is not None:
            e1_pred_io = e1_logits_io.argmax(1)                 # deep I/O
        else:
            # if no deep I/O head, fall back to E0 prediction (no gain possible)
            e1_pred_io = e0_pred

        # final I/O per-sample: E0 for early, E1 for late
        io_final_pred = torch.where(exit_mask2, e1_pred_io, e0_pred)

        # ---- I/O accuracies ----
        if exit_mask1.any():
            correct_at_exit[1] += (e0_pred[exit_mask1] == io_lbls[exit_mask1]).sum().item()

        if exit_mask2.any():
            # final I/O on late
            correct_io_late_final += (io_final_pred[exit_mask2] == io_lbls[exit_mask2]).sum().item()
            y_true_io_at_exit2.extend(io_lbls[exit_mask2].tolist())
            y_pred_io_at_exit2_final.extend(io_final_pred[exit_mask2].tolist())

            # E0 what-if on late subset
            correct_io_late_e0 += (e0_pred[exit_mask2] == io_lbls[exit_mask2]).sum().item()
            y_true_io_at_exit2_e0.extend(io_lbls[exit_mask2].tolist())
            y_pred_io_at_exit2_e0.extend(e0_pred[exit_mask2].tolist())

        # I/O force-all (E0 over all)
        correct_io_overall_e0 += (e0_pred == io_lbls).sum().item()

        # Collect for overall final I/O F1
        y_true_io_all.extend(io_lbls.tolist())
        y_pred_io_final_all.extend(io_final_pred.tolist())

        # ---- Scene metrics (only for late samples) ----
        if exit_mask2.any():
            raw_scene_pred = e1_logits_sc.argmax(1)             # [B]
            # buffers for raw @ exit2
            y_true_scene_at_exit2_raw.extend(sc_lbls[exit_mask2].tolist())
            y_pred_scene_at_exit2_raw.extend(raw_scene_pred[exit_mask2].tolist())

            # Post-hoc rule: if final I/O says Outdoor, map Scene→Street
            final_scene_pred = raw_scene_pred.clone()
            mask_outdoor_late = (io_final_pred == outdoor_label_int) & exit_mask2
            final_scene_pred[mask_outdoor_late] = street_label_int

            # final scene @ exit2 (late subset)
            y_true_scene_at_exit2_final.extend(sc_lbls[exit_mask2].tolist())
            y_pred_scene_at_exit2_final.extend(final_scene_pred[exit_mask2].tolist())

            # overall "scene_all" = late-only (consistent with your setup)
            y_true_scene_all.extend(sc_lbls[exit_mask2].tolist())
            y_pred_scene_all.extend(final_scene_pred[exit_mask2].tolist())

            # accuracy at exit2 (scene)
            correct_at_exit[2] += (final_scene_pred[exit_mask2] == sc_lbls[exit_mask2]).sum().item()

    # --- aggregate ---
    n1, n2 = exit_counts[1], exit_counts[2]
    total = total_samples

    # F1s
    f1_io_final_all_w   = f1_score(y_true_io_all, y_pred_io_final_all, average='weighted', zero_division=0) if y_true_io_all else 0.0
    f1_io_exit2_final_w = f1_score(y_true_io_at_exit2, y_pred_io_at_exit2_final, average='weighted', zero_division=0) if y_true_io_at_exit2 else 0.0
    f1_io_exit2_e0_w    = f1_score(y_true_io_at_exit2_e0, y_pred_io_at_exit2_e0, average='weighted', zero_division=0) if y_true_io_at_exit2_e0 else 0.0

    f1_scene_all_w        = f1_score(y_true_scene_all, y_pred_scene_all, average='weighted', zero_division=0) if y_true_scene_all else 0.0
    f1_scene_exit2_raw_w  = f1_score(y_true_scene_at_exit2_raw,  y_pred_scene_at_exit2_raw,  average='weighted', zero_division=0) if y_true_scene_at_exit2_raw else 0.0
    f1_scene_exit2_final_w= f1_score(y_true_scene_at_exit2_final,y_pred_scene_at_exit2_final,average='weighted', zero_division=0) if y_true_scene_at_exit2_final else 0.0

    return {
        "battery_level": battery_level,
        "exit_rates": {
            "exit1": (n1 / total * 100.0) if total else 0.0,
            "exit2": (n2 / total * 100.0) if total else 0.0,
        },
        "accuracy_at_exit": {
            "io@1_early":        (correct_at_exit[1] / n1) if n1 else 0.0,
            "io@2_final":        (correct_io_late_final / n2) if n2 else 0.0,
            "io@late_subset_e0": (correct_io_late_e0 / n2) if n2 else 0.0,
            "io@1_overall":      (correct_io_overall_e0 / total) if total else 0.0,
            "scene@2":           (correct_at_exit[2] / n2) if n2 else 0.0,
        },
        "f1_weighted": {
            "f1_io_final_all":     f1_io_final_all_w,
            "f1_io_at_exit2":      f1_io_exit2_final_w,   # final I/O on late
            "f1_io_at_exit2_e0":   f1_io_exit2_e0_w,      # E0 what-if on late
            "f1_scene_exit2_raw":   f1_scene_exit2_raw_w,
            "f1_scene_exit2_final": f1_scene_exit2_final_w,
            "f1_scene_all":         f1_scene_all_w,
        },
        "counts": {
            "total": total,
            "n_exit1": n1,
            "n_exit2": n2,
        }
    }

BEST_MODEL_PATH = "/kaggle/working/validation_runs/LR_0.0003_WD_5e-05_SCH_step_X113/best_model.pth"

model = DepthwiseEarlyExitCNN1D(
    if_train=False,
    train_shape=train_loader.dataset[0][0].shape,
    n_io=len(train_dataset.io_label_encoder.classes_),
    n_scene=len(train_dataset.scene_label_encoder.classes_)
).to(device)

# Load the weights of your chosen best model
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# You can test any battery levels you want
# battery_levels = np.linspace(0.0, 1.0, 11) # Example: test at 0, 0.25, 0.5, 0.75, 1.0
battery_levels = [0.3, 0.4, 0.6]
for b in battery_levels:
    results = evaluate_model_performance(model, test_loader, battery_level=b)
    
    # --- CORRECTED: Updated print statements ---
    print(f"--- Eval @ batt={b:.2f} | Rates: {results['exit_rates']} ---")
    print(f"Accuracies: {results['accuracy_at_exit']}")
    # print(f"F1 Weighted: {results['f1_weighted_at_exit']}")
    print("-"*50 + "\n")

--- Eval @ batt=0.30 | Rates: {'exit1': 50.99213777611381, 'exit2': 49.007862223886185} ---
Accuracies: {'io@1_early': 0.9720998531571219, 'io@2_final': 0.8819709702062644, 'io@late_subset_e0': 0.8113063407181055, 'io@1_overall': 0.8932983901160614, 'scene@2': 0.7203972498090145}
--------------------------------------------------

--- Eval @ batt=0.40 | Rates: {'exit1': 17.147135904155746, 'exit2': 82.85286409584425} ---
Accuracies: {'io@1_early': 0.990174672489083, 'io@2_final': 0.9154993221870764, 'io@late_subset_e0': 0.8732489832806145, 'io@1_overall': 0.8932983901160614, 'scene@2': 0.7783551739719837}
--------------------------------------------------

--- Eval @ batt=0.60 | Rates: {'exit1': 3.7251965555971545, 'exit2': 96.27480344440285} ---
Accuracies: {'io@1_early': 1.0, 'io@2_final': 0.9255298463931557, 'io@late_subset_e0': 0.8891697452848532, 'io@1_overall': 0.8932983901160614, 'scene@2': 0.804199883336574}
--------------------------------------------------



All-samples Exit2 classification report:
              precision    recall  f1-score   support

         Bar      0.551     0.719     0.624       828
     Hallway      0.780     0.764     0.772      1097
      Office      0.858     0.814     0.835      1884
      Street      0.838     0.766     0.800      1533

    accuracy                          0.775      5342
   macro avg      0.757     0.766     0.758      5342
weighted avg      0.789     0.775     0.779      5342



In [101]:
from sklearn.metrics import confusion_matrix, classification_report

@torch.no_grad()
def collect_scene_preds_exit2(model, loader, device, battery_level=1.0, force_exit2=True):
    """
    Returns:
        y_true: np.array of true scene labels
        y_pred: np.array of predicted scene labels at exit2
        used_idx: indices of samples actually evaluated (important if force_exit2=False)
    """
    model.eval()
    y_true, y_pred, used_idx = [], [], []
    idx = 0

    for batch in loader:
        # Your dataset order was (feats, io_lbls, sc_lbls) in many snippets
        if len(batch) == 3:
            feats, io_lbls, sc_lbls = batch
        else:
            # If your loader yields dicts or different tuples, adapt here
            raise ValueError("Expected (feats, io_lbls, sc_lbls) from the loader.")

        feats = feats.to(device, non_blocking=True).float()
        sc_lbls = sc_lbls.to(device, non_blocking=True).long()

        if force_exit2:
            # Trick: set if_train=True temporarily to always fetch e1sc (exit2 scene logits)
            was_train_flag = model.if_train
            model.if_train = True
            out_list, _ = model(feats, battery_level=battery_level)
            model.if_train = was_train_flag

            e1sc = out_list[1]          # [B, n_scene]
            pred = e1sc.argmax(dim=1)   # scene prediction at exit2 for all samples

            y_true.append(sc_lbls.cpu().numpy())
            y_pred.append(pred.cpu().numpy())
            used_idx.extend(range(idx, idx + sc_lbls.shape[0]))

        else:
            # Policy-based: only include samples that actually go deep in normal inference
            was_train_flag = model.if_train
            model.if_train = False
            out_list, exit_id = model(feats, battery_level=battery_level)
            model.if_train = was_train_flag

            # exit_id can be scalar or per-sample; in your code it’s a scalar for the whole batch.
            if exit_id == 2:
                # out_list = [io_final, e1sc]
                e1sc = out_list[1]
                pred = e1sc.argmax(dim=1)
                y_true.append(sc_lbls.cpu().numpy())
                y_pred.append(pred.cpu().numpy())
                used_idx.extend(range(idx, idx + sc_lbls.shape[0]))
            # else: all samples exited early → nothing added (this is expected)

        idx += sc_lbls.shape[0]

    if len(y_true) == 0:
        # No samples reached exit2 in policy mode
        return np.array([]), np.array([]), np.array(used_idx, dtype=int)

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    return y_true, y_pred, np.array(used_idx, dtype=int)


def plot_confusion_heatmap(y_true, y_pred, class_names, normalize=True, title="Scene @ Exit2"):
    """
    Draws a single heatmap using matplotlib (no seaborn).
    normalize=True shows per-row (true class) normalization.
    """
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
    if normalize:
        cm = cm.astype(np.float64)
        row_sums = cm.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1.0
        cm = cm / row_sums

    fig, ax = plt.subplots(figsize=(6, 5), dpi=120)
    im = ax.imshow(cm, aspect='auto')
    ax.set_title(title)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")

    ax.set_xticks(range(len(class_names)))
    ax.set_yticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.set_yticklabels(class_names)

    # Annotate cells
    h, w = cm.shape
    for i in range(h):
        for j in range(w):
            val = cm[i, j]
            text = f"{val:.2f}" if normalize else f"{int(val)}"
            ax.text(j, i, text, va='center', ha='center', fontsize=9)

    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    return fig

class_names = list(test_loader.dataset.scene_label_encoder.classes_)  # or your own list

BEST_MODEL_PATH = "/kaggle/working/validation_runs/LR_0.0003_WD_5e-05_SCH_step_X111/best_model.pth"

model = DepthwiseEarlyExitCNN1D(
    if_train=False,
    train_shape=train_loader.dataset[0][0].shape,
    n_io=len(train_dataset.io_label_encoder.classes_),
    n_scene=len(train_dataset.scene_label_encoder.classes_)
).to(device)

# Load the weights of your chosen best model
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# 1) FORCE exit2: evaluate scene head on all samples
y_true_all, y_pred_all, idx_all = collect_scene_preds_exit2(model, test_loader, device, battery_level=1.0, force_exit2=True)
print("All-samples Exit2 classification report:")
print(classification_report(y_true_all, y_pred_all, target_names=class_names, digits=3))
fig1 = plot_confusion_heatmap(y_true_all, y_pred_all, class_names, normalize=True, title="Scene @ Exit2 (forced for all samples)")
plt.show()

In [32]:
@torch.no_grad()
def collect_scene_preds_exit2(model, loader, device, battery_level=1.0, force_exit2=True):
    """
    Returns:
        y_true: np.array of true scene labels
        y_pred: np.array of predicted scene labels at exit2
        used_idx: indices of samples actually evaluated (important if force_exit2=False)
    """
    model.eval()
    y_true, y_pred, used_idx = [], [], []
    idx = 0

    for batch in loader:
        # Your dataset order was (feats, io_lbls, sc_lbls) in many snippets
        if len(batch) == 3:
            feats, io_lbls, sc_lbls = batch
        else:
            # If your loader yields dicts or different tuples, adapt here
            raise ValueError("Expected (feats, io_lbls, sc_lbls) from the loader.")

        feats = feats.to(device, non_blocking=True).float()
        sc_lbls = sc_lbls.to(device, non_blocking=True).long()

        if force_exit2:
            # Trick: set if_train=True temporarily to always fetch e1sc (exit2 scene logits)
            was_train_flag = model.if_train
            model.if_train = True
            out_list, _ = model(feats, battery_level=battery_level)
            model.if_train = was_train_flag

            e1sc = out_list[1]          # [B, n_scene]
            pred = e1sc.argmax(dim=1)   # scene prediction at exit2 for all samples

            y_true.append(sc_lbls.cpu().numpy())
            y_pred.append(pred.cpu().numpy())
            used_idx.extend(range(idx, idx + sc_lbls.shape[0]))

        else:
            # Policy-based: only include samples that actually go deep in normal inference
            was_train_flag = model.if_train
            model.if_train = False
            out_list, exit_id = model(feats, battery_level=battery_level)
            model.if_train = was_train_flag

            # exit_id can be scalar or per-sample; in your code it’s a scalar for the whole batch.
            if exit_id == 2:
                # out_list = [io_final, e1sc]
                e1sc = out_list[1]
                pred = e1sc.argmax(dim=1)
                y_true.append(sc_lbls.cpu().numpy())
                y_pred.append(pred.cpu().numpy())
                used_idx.extend(range(idx, idx + sc_lbls.shape[0]))
            # else: all samples exited early → nothing added (this is expected)

        idx += sc_lbls.shape[0]

    if len(y_true) == 0:
        # No samples reached exit2 in policy mode
        return np.array([]), np.array([]), np.array(used_idx, dtype=int)

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    return y_true, y_pred, np.array(used_idx, dtype=int)


def plot_confusion_heatmap(y_true, y_pred, class_names, normalize=True, title="Scene @ Exit2"):
    """
    Draws a single heatmap using matplotlib (no seaborn).
    normalize=True shows per-row (true class) normalization.
    """
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
    if normalize:
        cm = cm.astype(np.float64)
        row_sums = cm.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1.0
        cm = cm / row_sums

    fig, ax = plt.subplots(figsize=(6, 5), dpi=120)
    im = ax.imshow(cm, aspect='auto')
    ax.set_title(title)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")

    ax.set_xticks(range(len(class_names)))
    ax.set_yticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.set_yticklabels(class_names)

    # Annotate cells
    h, w = cm.shape
    for i in range(h):
        for j in range(w):
            val = cm[i, j]
            text = f"{val:.2f}" if normalize else f"{int(val)}"
            ax.text(j, i, text, va='center', ha='center', fontsize=9)

    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    return fig


# ---------------------- HOW TO USE --------------------------------
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# class_names = list(test_loader.dataset.scene_label_encoder.classes_)  # or your own list
#
# # 1) FORCE exit2: evaluate scene head on all samples
# y_true_all, y_pred_all, idx_all = collect_scene_preds_exit2(
#     model, test_loader, device, battery_level=1.0, force_exit2=True
# )
# print("All-samples Exit2 classification report:")
# print(classification_report(y_true_all, y_pred_all, target_names=class_names, digits=3))
# fig1 = plot_confusion_heatmap(y_true_all, y_pred_all, class_names, normalize=True,
#                               title="Scene @ Exit2 (forced for all samples)")
# plt.show()

In [ ]:
def evaluate_sensor_subset(model, loader, battery_level, sensor_slices, active_sensors):
    model.eval()
    model.if_train = False

    # Grab one batch just to get C and T
    T, C = loader.dataset[0][0].shape

    mask = torch.zeros(1, 1, C, device=device)
    for i in active_sensors:
        c0, c1 = sensor_slices[i]
        mask[..., c0:c1] = 1
        
    # Build hard mask on channel dim: shape (1, C, 1)
    # mask = torch.zeros(1, C, 1, device=device)
    # for i in active_sensors:
    #     c0, c1 = sensor_slices[i]
    #     mask[:, c0:c1, :] = 1

    total = 0
    exit_counts = {1: 0, 2: 0, 3: 0}
    correct_exit = {1: 0, 2: 0, 3: 0}
    y_true = {1: [], 2: [], 3: []}
    y_pred = {1: [], 2: [], 3: []}

    with torch.no_grad():
        for feats, io_lbls, sc_lbls in loader:
            # feats: (B, T, C)
            feats = feats.float().to(device) * mask   # still (B, T, C)
            io_lbls = io_lbls.view(-1).to(device)
            sc_lbls = sc_lbls.view(-1).to(device)
            bsz = feats.size(0)

            batt = torch.full((bsz,), battery_level, dtype=torch.float32, device=device)
            outs, exit_idx = model(feats, battery_level=batt)
            exit_idx = int(exit_idx)

            total += bsz
            exit_counts[exit_idx] += bsz

            if exit_idx == 1:
                preds = outs[0].argmax(dim=1); labels = io_lbls
            elif exit_idx == 2:
                preds = outs[1].argmax(dim=1); labels = sc_lbls
            else:
                preds = outs[2].argmax(dim=1); labels = sc_lbls

            correct_exit[exit_idx] += (preds == labels).sum().item()
            y_true[exit_idx].extend(labels.cpu().tolist())
            y_pred[exit_idx].extend(preds.cpu().tolist())

    # summarize
    exit_rates = {e: exit_counts[e]/total*100 for e in exit_counts}
    accuracy_at_exit = {
        e: correct_exit[e]/exit_counts[e] if exit_counts[e] else 0.0
        for e in exit_counts
    }
    overall_acc = sum(correct_exit.values()) / total

    f1_at_exit = {}
    for e in exit_counts:
        if exit_counts[e]:
            f1_at_exit[e] = f1_score(
                y_true[e], y_pred[e], average="macro", zero_division=0
            )
        else:
            f1_at_exit[e] = 0.0

    all_true = sum(y_true.values(), [])
    all_pred = sum(y_pred.values(), [])
    overall_f1 = (
        f1_score(all_true, all_pred, average="macro", zero_division=0)
        if total else 0.0
    )

    return {
        "active_sensors": active_sensors,
        "exit_rates":        exit_rates,
        "accuracy_at_exit":  accuracy_at_exit,
        "overall_acc":       overall_acc,
        "f1_at_exit":        f1_at_exit,
        "overall_f1":        overall_f1
    }

# FIX this

In [47]:
BEST_MODEL_PATH = "/kaggle/working/validation_runs/LR_0.0003_WD_5e-05_SCH_step_DROP_0.0_X100/best_model.pth"

model = DepthwiseEarlyExitCNN1D(
    if_train=False,
    train_shape=train_loader.dataset[0][0].shape,
    n_io=len(train_dataset.io_label_encoder.classes_),
    n_scene=len(train_dataset.scene_label_encoder.classes_),
    drop_p=hparams["drop_p"]
).to(device)

# Load the weights of your chosen best model
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# battery_levels = np.linspace(0, 1, 11)

sensor_slices = [
    ( 0,  3),  # Acc: channels 0,1,2
    ( 3,  6),  # Mag: 3,4,5
    ( 6,  9),  # Gyro:6,7,8
    ( 9, 10),  # Press:9
    (10, 11),  # Temp:10
]
sensors_name = ["Acc", "Mag", "Gyro", "Press", "Temp"] # check the order of sensors
keep_sensors = [0, 1]

# keep only sensors[0] and sensors[1]
result = evaluate_sensor_subset(
    model=model,
    loader=test_loader,
    battery_level=1.0,
    sensor_slices=sensor_slices,
    active_sensors=list(range(len(sensor_slices)))
)

# Build list of kept sensor names
kept = [sensors_name[i] for i in keep_sensors]
print("Kept sensors:", ", ".join(kept))
print(f"Exit rates: {result['exit_rates']}")
print(f"Accuracy at exits: {result['accuracy_at_exit']}")
print(f"Overall accuracy: {result['overall_acc']:.3f}")

NameError: name 'evaluate_sensor_subset' is not defined

In [ ]:
import itertools
import json
import os

train_shape = train_ds[0][0].shape

model = DepthwiseEarlyExitCNN1D(
    if_train=True,
    train_shape=train_shape,
    n_io=len(train_ds.io_label_encoder.classes_),
    n_scene=len(train_ds.scene_label_encoder.classes_)
).to(device)

# Load the weights of your chosen best model
model.load_state_dict(torch.load(BEST_MODEL_PATH))

sensor_slices = [(0,6), (6,9), (9,12), (12,13), (13,14)]
sensor_indices = [0, 1, 2, 3, 4]
sensor_names = ["Acc", "Mag", "Gyro", "Press", "Temp"]

# Generate all non-empty subsets (combinations) of sensor indices
all_combinations = [
    list(combo)
    for r in range(1, len(sensor_indices) + 1)
    for combo in itertools.combinations(sensor_indices, r)
]

# Evaluate each combination and collect results
results = []
i = 1
for combo in all_combinations:
    print(f"Iteration {i}/{len(all_combinations)} Evaluating combo: {combo}")
    # Evaluate the model on this subset
    res = evaluate_sensor_subset(
        model=model,
        loader=test_loader,
        battery_level=0.5,
        sensor_slices=sensor_slices,
        active_sensors=combo
    )
    # Map indices to names for readability
    res["active_sensor_names"] = [sensor_names[i] for i in combo]
    results.append(res)

    i += 1

# Define output filepath
output_path = "/kaggle/working/b_1.0.json"

# Save results to JSON
with open(output_path, "w") as f:
    json.dump(results, f, indent=2)

print(f"Saved evaluation results for {len(results)} combinations")

## Evaluating performance

In [51]:
def count_parameters(model):
    """Counts the total number of trainable parameters in a PyTorch model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

BEST_MODEL_PATH = "/kaggle/working/validation_runs/LR_0.0003_WD_5e-05_SCH_step_X113/best_model.pth"

model = DepthwiseEarlyExitCNN1D(
    if_train=False,
    train_shape=train_loader.dataset[0][0].shape,
    n_io=len(train_dataset.io_label_encoder.classes_),
    n_scene=len(train_dataset.scene_label_encoder.classes_)
).to(device)

# Load the weights of your chosen best model
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Now, call the function to get the count
total_params = count_parameters(model)
print(f"Total Trainable Parameters: {total_params:}")

estimated_size = total_params * 4 / 1000 # FP32 precision
print(f"Estimated Size: {estimated_size:,} KB")

Total Trainable Parameters: 5676
Estimated Size: 22.704 KB


In [ ]:
from torchinfo import summary

model = DepthwiseEarlyExitCNN1D(
    if_train=False,
    train_shape=train_loader.dataset[0][0].shape,
    n_io=len(train_dataset.io_label_encoder.classes_),
    n_scene=len(train_dataset.scene_label_encoder.classes_)
).to(device)

# Print summary
summary(
    model, 
    input_size=(1, 11, 600), 
    col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"],
    verbose=2,  # Detailed output
    device=device
)

# TODO
- Data Augmentation for Time-Series Data
- attention mechanism

# TO DELETE THE /kaggle/working/ FILES

In [48]:
import shutil
import os

directory_path = "/kaggle/working/validation_runs/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
    print(f"Deleted directory {directory_path}")
else:
    print(f"Directory {directory_path} not found")

Deleted directory /kaggle/working/validation_runs/
